In [1]:
import tensorflow

In [2]:
from tensorflow.keras.datasets import mnist

ERROR! Session/line number was not unique in database. History logging moved to new session 142


In [3]:
((trainData, trainLabels), (testData, testLabels)) = mnist.load_data()

In [4]:
trainData = trainData.reshape((trainData.shape[0], 28, 28, 1))
testData = testData.reshape((testData.shape[0], 28, 28, 1))

In [5]:
trainData = trainData.astype("float32") / 255.0
testData = testData.astype("float32") / 255.0

In [6]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report

In [7]:
le = LabelBinarizer()
trainLabels = le.fit_transform(trainLabels)
testLabels = le.transform(testLabels)

some model parameters

In [8]:
INIT_LR = 1e-3
EPOCHS = 10
BS = 128


building model

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [10]:
model = Sequential()
inputShape = (28, 28, 1)
model.add(Conv2D(32, (5,5), padding="same", input_shape= inputShape))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))
model.add(Dropout(0.5))

model.add(Dense(64))
model.add(Activation("relu"))
model.add(Dropout(0.5))

model.add(Dense(64))
model.add(Activation("relu"))
model.add(Dropout(0.25))

model.add(Dense(10))
model.add(Activation("softmax"))

ERROR! Session/line number was not unique in database. History logging moved to new session 143


In [11]:
from tensorflow.keras.optimizers import Adam

ERROR! Session/line number was not unique in database. History logging moved to new session 144


In [12]:
opt = Adam(lr=INIT_LR)

In [13]:
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [14]:
H = model.fit(trainData, trainLabels, validation_data=(testData, testLabels),
	batch_size=BS, epochs=EPOCHS, verbose=1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10


60000/60000 [==============================] - 45s 751us/sample - loss: 0.8667 - accuracy: 0.7015 - val_loss: 0.1106 - val_accuracy: 0.9706
Epoch 2/10
60000/60000 [==============================] - 43s 721us/sample - loss: 0.3262 - accuracy: 0.9057 - val_loss: 0.0709 - val_accuracy: 0.9820
Epoch 3/10
60000/60000 [==============================] - 48s 794us/sample - loss: 0.2443 - accuracy: 0.9309 - val_loss: 0.0540 - val_accuracy: 0.9856
Epoch 4/10
60000/60000 [==============================] - 55s 918us/sample - loss: 0.2066 - accuracy: 0.9408 - val_loss: 0.0497 - val_accuracy: 0.9874
Epoch 5/10
60000/60000 [==============================] - 50s 841us/sample - loss: 0.1860 - accuracy: 0.9478 - val_loss: 0.0416 - val_accuracy: 0.9885
Epoch 6/10
60000/60000 [==============================] - 49s 814us/sample - loss: 0.1697 - accuracy: 0.9522 - val_loss: 0.0392 - val_accuracy: 0.9889
Epoch 7/10
60000/60000 [==============================] - 46s 763us/sample - loss: 0.1558 - accuracy: 0.9

In [102]:
%store -r extracted_digits

In [103]:
import cv2
import numpy as np
present = 0
from tensorflow.keras.preprocessing.image import img_to_array
board = np.zeros((9,9), dtype='int')
for i in range(81):
    if extracted_digits[i] is not None:
        present+=1
        value = cv2.resize(extracted_digits[i], (28,28))
        value = value.astype("float") / 255.00
        value = img_to_array(value)
        value = np.expand_dims(value, axis=0)
        prediction = model.predict(value) 
        board[i // 9][i%9] = np.argmax(prediction)
print( present, "number of cells which are non-empty")

31 number of cells which are non-empty


In [106]:
for i in range(9):
    for j in range(9):
        print(board[i][j], end = "  ")
    print('\n')

0  0  4  1  9  9  0  7  0  

1  0  0  0  0  3  0  3  0  

0  0  9  0  0  0  4  0  0  

0  0  6  7  0  0  0  3  0  

6  0  2  0  0  0  2  0  0  

0  1  0  0  0  0  3  0  0  

0  0  8  0  0  0  7  0  0  

0  5  0  6  0  0  0  0  3  

6  0  0  5  8  9  3  0  0  

